# Mirror repeat analysis of T2T genome
## Table of contents <a name="TOC"></a>
1. [Expected number of repeats and AG segments](#Expected_number)
    1. [Expected number of AG segments](#Expected_number_segm)
    2. [Expected number of purine MR](#Expected_number_repeats)
    3. [Number of repeats in AG segments](#Number_repeats_in_segments)
    4. [Expected number of repeats in AG segments](#Expected_number_repeats_in_segments)
2. [Structure of the mirror repeats](#structure)
    1. [Patterns](#patterns)
    2. [Types of repeats](#repeat_type)
3. [Annotation](#annotation)
    1. [RepeatMasker database](#RM)
    2. [Genes](#genes)
    3. [Introns](#introns)
    4. [Enhancers](#enhancers)
    5. [Promoters](#promoters)
    6. [File for heatmap with contributions of each features](#contribution)
    7. [TE](#TE)
    8. [ISTAT](#istat)
4. [Conservation](#conservation)

In [1]:
import pandas as pd
import numpy as np
import re
import random
import gzip
from natsort import index_natsorted

In [2]:
from Bio import pairwise2
from Bio.Seq import Seq 

C:\Users\User\anaconda3\lib\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [3]:
from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio import SeqIO 
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [4]:
chrom_list_XY = list(range(1,23)) + ['X', 'Y']
chr_list_XY = ['chr'+str(i) for i in chrom_list_XY]

Download T2T genome: T2T-CHM13v2.0 https://www.ncbi.nlm.nih.gov/assembly/GCA_009914755.4#/def

In [5]:
CHM13_genome = dict()
chr_range = 1
chr_list = list(range(chr_range,23))
with open('../../../TtoT_genome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna') as fasta_file:
    for sequence in SimpleFastaParser(fasta_file):
        chrom = re.split('chromosome |,', sequence[0])[1]
        if chrom in [str(chrom) for chrom in chr_list]:
            CHM13_genome[int(chrom)] = sequence[1].upper()
            print(chrom, end=', ')
        elif chrom in ['X', 'Y']:
            CHM13_genome[chrom] = sequence[1].upper()
            print(chrom, end=', ')

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, X, Y, 

In [20]:
#input files
MRpur = pd.read_csv('data/MRpurine_HumanGenome.csv', sep='\t')
AGsegm = pd.read_csv('data/AGsegments_HumanGenome.out', sep='\t')

## Expected number of repeats and segments

<a name='Expected_number'></a>
[Return to Table of Contents](#TOC)

### Expected number of AG segments

<a name='Expected_number_segm'></a>
[Return to Table of Contents](#TOC)

In [6]:
chrom_list_XY = list(range(1,23)) + ['X', 'Y']
base_counts = dict()
for chrom in chrom_list_XY:
    base_counts[chrom] = pd.Series([CHM13_genome[chrom].count('A'), CHM13_genome[chrom].count('T'), CHM13_genome[chrom].count('G'), CHM13_genome[chrom].count('C')], index = ['A', 'T', 'G', 'C'])
base_counts = pd.concat(base_counts).unstack()

In [7]:
base_counts

,A,T,G,C
1,73600418,71705495,52064401,51017014
2,72424400,72614972,48914580,48742800
3,61130447,60661558,39576837,39737106
4,59694025,60069053,37006775,36805092
5,54881762,55211290,35979055,35973332
6,51932667,52011787,34148104,34034070
7,47481945,47705013,32697146,32683324
8,43714844,43699677,29464192,29380618
9,44172441,44170480,31213290,31061036
10,39354993,39425966,28062985,27914190


In [8]:
base_counts.sum(axis=0) / base_counts.sum().sum()

A    0.296372
T    0.296089
G    0.204255
C    0.203284
dtype: float64

In [9]:
pA = 0.296372
pT = 0.296089
pG = 0.204255
pC = 0.203284
pR = pA + pG
genome_len = 3117275501
n_expected = []
for i in range(20, 4535):
    num = (pR**(i))*genome_len
    n_expected.append(int(round(num, 0)))
Len = list(range(20,4535))
d = {'Len': Len, 'n_expected': n_expected}
df = pd.DataFrame(data=d)
df.to_csv('ExpectedNumber_AGsegments_HumanGenome.csv', sep='\t', index=False)
df.head()

,Len,n_expected
0,20,3048
1,21,1526
2,22,764
3,23,382
4,24,191


### Expected number of purine MR

<a name='Expected_number_repeats'></a>
[Return to Table of Contents](#TOC)

- generating chromosomes with nucleotides probabilities

In [6]:
base_counts = dict()
for chrom in chrom_list_XY:
    base_counts[chrom] = pd.Series([CHM13_genome[chrom].count('A'), CHM13_genome[chrom].count('T'), CHM13_genome[chrom].count('G'), CHM13_genome[chrom].count('C')], index = ['A', 'T', 'G', 'C'])
base_counts = pd.concat(base_counts).unstack()

In [7]:
base_counts.sum(axis=0) / base_counts.sum().sum()

A    0.296372
T    0.296089
G    0.204255
C    0.203284
dtype: float64

In [8]:
# Note: using larger integers increases time to generate sequence
def DNA(length, A_frac = 296, T_frac = 296, G_frac = 204, C_frac = 204):
    return ''.join(random.choice('A'*A_frac+'T'*T_frac+'G'*G_frac+'C'*C_frac) for _ in range(length))

In [11]:
chrom_lengths = pd.Series([len(CHM13_genome[seq]) for seq in CHM13_genome], index = chrom_list_XY)

In [13]:
random_genome = {}
for chrom in chrom_list_XY:
    print('\r' + str(chrom), end=' ')
    random_genome[chrom] = DNA(chrom_lengths[chrom])

Y  

In [18]:
for chrom in chrom_list_XY:
    outfile = 'data/outputs/chr' + str(chrom) + '_generated.fasta'
    with open(outfile, 'w') as fout:
        fout.write('>chr'+str(chrom) + ' generated\n' + random_genome[chrom] + '\n')

### Number of repeats in AG segments

<a name='Number_repeats_in_segments'></a>
[Return to Table of Contents](#TOC)

In [58]:
outfile = pd.DataFrame(columns = ['chr', 'start1', 'start2', 'end', 'lsh', 'dist', 'first shoulder', 'second shoulder', 'repeat_seq', 'start_segm', 'end_segm', 'len_segm'])

pd.options.mode.chained_assignment = None

for Chr in chr_list_XY:
    print(Chr, end=' ')
    MRpur_chr = MRpur[MRpur['chr']==Chr]
    MRpur_chr = MRpur_chr.reset_index(drop=True)
    AGsegm_chr = AGsegm[AGsegm['chr']==Chr]
    AGsegm_chr = AGsegm_chr[AGsegm_chr['len']>=40]
    AGsegm_chr = AGsegm_chr.reset_index(drop=True)
    
    
    segm = 0
    for rep in range(0, len(MRpur_chr)):
        rep_start = MRpur_chr.iloc[rep]['start1']
        rep_end = MRpur_chr.iloc[rep]['end']
        segm_start = AGsegm_chr.iloc[segm]['start']
        segm_end = AGsegm_chr.iloc[segm]['end']
        
        while (rep_start > segm_end) and (segm < len(AGsegm_chr) - 1):
            segm += 1
            segm_start = AGsegm_chr.iloc[segm]['start']
            segm_end = AGsegm_chr.iloc[segm]['end']
            
        if segm >= len(AGsegm_chr):
            break
            
        if (rep_start >= segm_start) and (rep_end <= segm_end):
            repeat = MRpur_chr.iloc[[rep]]
            repeat['start_segm'] = segm_start
            repeat['end_segm'] = segm_end
            repeat['len_segm'] = segm_end - segm_start
            outfile = pd.concat([outfile, repeat], ignore_index = True)
        
outfile.to_csv('data/outputs/MRpur_inAGsegm.csv', sep='\t', index=False)
outfile.head()

chr1 chr2 chr3 chr4 chr5 chr6 chr7 chr8 chr9 chr10 chr11 chr12 chr13 chr14 chr15 chr16 chr17 chr18 chr19 chr20 chr21 chr22 chrX chrY 

,chr,start1,start2,end,lsh,dist,first shoulder,second shoulder,repeat_seq,start_segm,end_segm,len_segm
0,chr1,73343,73369,73379,10,16,AGAGAGAGAA,AAGAGAGAGA,AGAGAGAGAAAGAGAAAGAAAGAAAGAAGAGAGAGA,73332,73506,174
1,chr1,73376,73401,73411,10,15,AGAAAGAAAG,GAAAGAAAGA,AGAAAGAAAGGGAGAGAAAGAAAAAGAAAGAAAGA,73332,73506,174
2,chr1,73390,73412,73429,17,5,AGAAAGAAAAAGAAAGA,AGAAAGAAAAAGAAAGA,AGAAAGAAAAAGAAAGAAAGAAAGAAAGAAAAAGAAAGA,73332,73506,174
3,chr1,73398,73430,73451,21,11,AAAGAAAGAAAGAAAGAAAGA,AGAAAGAAAGAAAGAAAGAAA,AAAGAAAGAAAGAAAGAAAGAAAAAGAAAGAAAGAAAGAAAGAAAG...,73332,73506,174
4,chr1,73438,73452,73465,13,1,AGAAAGAAAGAAA,AAAGAAAGAAAGA,AGAAAGAAAGAAAGAAAGAAAGAAAGA,73332,73506,174


### Expected number of repeats in AG segments

<a name='Expected_number_repeats_in_segments'></a>
[Return to Table of Contents](#TOC)
- generating AG segments in chromosomes with AG content

In [66]:
for chrom in chrom_list_XY:
    print(chrom, end = ' ')
    seq_chr = CHM13_genome[chrom]
    AGsegm_chr = AGsegm[AGsegm['chr']=='chr'+str(chrom)]
    AGsegm_chr = AGsegm_chr.reset_index(drop=True)
    
    outfile = "data/outputs/chr" + str(chrom) + "_generatedAGsegm.fasta"
    with open(outfile, 'w') as fout:
        fout.write(">" + Chr +" with generated AG segments\n")
        ch = 0
        for row in range(0, len(AGsegm_chr)):
            segm_start = AGsegm_chr.iloc[segm]['start']
            segm_end = AGsegm_chr.iloc[segm]['end']
            fout.write(seq_chr[ch:segm_start])
            
            segm_seq = seq_chr[segm_start:segm_end]
            nA = segm_seq.count('A')
            probA = nA/(segm_end-segm_start)
            probG = 1 - probA
            
            fout.write(''.join(list(np.random.choice(['A', 'G'], segm_end-segm_start, p=[probA, probG]))))
            
            ch = segm_end
        
        fout.write(seq_chr[ch:])

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 X Y 

## Structure of the mirror repeats

<a name='structure'></a>
[Return to Table of Contents](#TOC)

### Patterns in MR

<a name='patterns'></a>
[Return to Table of Contents](#TOC)
- finding patterns in purine MR

In [8]:
patterns = []
complexity = []
for i in range(0, len(MRpur)):
    
    # pattern was not found
    flag = False
    
    seq = MRpur.iloc[i]['repeat_seq']
    if ('A' not in seq):
        patterns.append('G')
        complexity.append('1')
        continue
    if ('G' not in seq):
        patterns.append('A')
        complexity.append('1')
        continue
    subseqs = re.findall(r'([AG]{2,}?)\1+', seq)
    for pattern in subseqs:
        if ('G' not in pattern) or ('A' not in pattern):
            continue
        seq_for_align = pattern * (len(seq)//len(pattern))
        score = pairwise2.align.globalms(seq, seq_for_align, 1, 0, -0.1, -0.1, score_only=True)
        percent = round((score / len(seq)), 2)
        if percent >= 0.85:
            # repeat consists of patterns 
            patterns.append(pattern)
            # guess that complexity is length of the pattern
            num = len(pattern)
            complexity.append(num)
            flag = True
            break
    if flag == False:
        patterns.append('-')
        num = len(seq)
        complexity.append(num)

MRpur['pattern'] = patterns
MRpur['complexity'] = complexity
MRpur.to_csv('MRpur_patterns', sep='\t', index = False)
MRpur.head()

,chr,start1,start2,end,lsh,dist,first shoulder,second shoulder,repeat_seq,pattern,complexity
0,chr1,50042,50060,50071,11,7,GGAGGGAGGGA,AGGGAGGGAGG,GGAGGGAGGGAGGCAGAAAGGGAGGGAGG,GGAG,4
1,chr1,50059,50076,50086,10,7,AAGGGAGGGA,AGGGAGGGAA,AAGGGAGGGAGGCAGAAAGGGAGGGAA,-,27
2,chr1,50082,50096,50107,11,3,GGAAGGAAGGA,AGGAAGGAAGG,GGAAGGAAGGAGCAAGGAAGGAAGG,GGAA,4
3,chr1,50097,50108,50119,11,0,GGAAGGAAGGA,AGGAAGGAAGG,GGAAGGAAGGAAGGAAGGAAGG,GGAA,4
4,chr1,73343,73369,73379,10,16,AGAGAGAGAA,AAGAGAGAGA,AGAGAGAGAAAGAGAAAGAAAGAAAGAAGAGAGAGA,AG,2


### Types of repeats

<a name='repeat_type'></a>
[Return to Table of Contents](#TOC)

In [10]:
MR_type = []
for i in range(0, len(MRpur)):
    if MRpur.iloc[i]['lsh'] < 22:
        if MRpur.iloc[i]['pattern'] == '-':
            MR_type.append('short_complex')
        else:
            MR_type.append('STR')
    else:
        if MRpur.iloc[i]['pattern'] == '-':
            MR_type.append('long_complex')
        else:
            if len(MRpur.iloc[i]['pattern']) < 22:
                MR_type.append('long_TR')
            else:
                MR_type.append('long_complex')
MRpur['repeat_type'] = MR_type
#MRpur.to_csv('MRpur_TypeRepeats.csv', '\t', index=False)
MRpur.head()

,chr,start1,start2,end,lsh,dist,first shoulder,second shoulder,repeat_seq,pattern,complexity,repeat_type
0,chr1,50042,50060,50071,11,7,GGAGGGAGGGA,AGGGAGGGAGG,GGAGGGAGGGAGGCAGAAAGGGAGGGAGG,GGAG,4,STR
1,chr1,50059,50076,50086,10,7,AAGGGAGGGA,AGGGAGGGAA,AAGGGAGGGAGGCAGAAAGGGAGGGAA,-,27,short_complex
2,chr1,50082,50096,50107,11,3,GGAAGGAAGGA,AGGAAGGAAGG,GGAAGGAAGGAGCAAGGAAGGAAGG,GGAA,4,STR
3,chr1,50097,50108,50119,11,0,GGAAGGAAGGA,AGGAAGGAAGG,GGAAGGAAGGAAGGAAGGAAGG,GGAA,4,STR
4,chr1,73343,73369,73379,10,16,AGAGAGAGAA,AAGAGAGAGA,AGAGAGAGAAAGAGAAAGAAAGAAAGAAGAGAGAGA,AG,2,STR


## Annotation

<a name='annotation'></a>
[Return to Table of Contents](#TOC)

In [23]:
# df is table with annotation (name columns: #chrom, thickStart, thickEnd, feature - class of element, *Name - name of element)
# MR - repeats for annotation (columns: start1 - origin coordinate of the first arm, end - end coordinate of the second arm)
# chr_list - list of chromosomes names
# ignore_names - if True there are no names of elements (only features f. ex. exon or CDS) if False there are names of element (f. ex. AluSx)
def annotation(df, MR, chr_list, ignore_names):
    def closest_value(input_list, input_value):
        arr = np.asarray(input_list)
        i = (np.abs(arr - input_value)).argmin()
        return i
    
    # feature - column name, ind - index of row in df
    def write_features(MR, feature, value, ind):
        if MR.at[ind, feature] == '-':
            MR.at[ind, feature] = value
        else:
            v0 = MR.at[ind, feature]
            if type(v0) == list:
                v0 = str(v0)
            if type(value) != list:
                value = '; ' + value
            else:
                v0 = v0.strip('][').split(', ')
            MR.at[ind, feature] = v0 + value
        return MR
    
    # index row in MR
    ind = 0
    for c in chr_list:
        print(c, end=' ')
        df_chr = df[df['#chrom']==c]
        MR_chr = MR[MR['chr']==c]
        
        if len(df_chr) == 0:
            continue

        repeats_starts = MR_chr['start1'].tolist()
        df_starts = df_chr['thickStart'].tolist()
        repeats_ends = MR_chr['end'].tolist()
        df_ends = df_chr['thickEnd'].tolist()
        df_features = df_chr['feature'].tolist()
        if ignore_names == False:
            df_name = df_chr['Name'].tolist()

        for rep in range(0, len(repeats_starts)):
            # for considering number of chromosome (miss rows for preceding chromosome)
            ind_MR = rep+ind
            #search the closest number and write index
            if __name__ == "__main__" :
                num=closest_value(df_starts,repeats_starts[rep])
                if (repeats_starts[rep] >= df_ends[num]) | (repeats_ends[rep] <= df_starts[num]):
                    pass
                elif (repeats_starts[rep] >= df_starts[num]) & (repeats_ends[rep] <= df_ends[num]):
                    MR = write_features(MR, 'feature', df_features[num], ind_MR)
                    if ignore_names == False:
                        MR = write_features(MR, 'name', df_name[num], ind_MR)
                    MR = write_features(MR, 'position', 'whole', ind_MR)
                    MR = write_features(MR, 'feature_coordinates', [df_starts[num], df_ends[num]], ind_MR)
                else:
                    rep_lsh = repeats_ends[rep] - repeats_starts[rep]
                    if (df_starts[num] < repeats_starts[rep]) & ((df_ends[num] - repeats_starts[rep]) > (rep_lsh // 2)):
                        MR = write_features(MR, 'feature', df_features[num], ind_MR)
                        if ignore_names == False:
                            MR = write_features(MR, 'name', df_name[num], ind_MR)
                        MR = write_features(MR, 'position', 'part', ind_MR)
                        MR = write_features(MR, 'feature_coordinates', [df_starts[num], df_ends[num]], ind_MR)
                    elif (df_starts[num] > repeats_starts[rep]) & ((repeats_ends[rep] - df_starts[num]) > (rep_lsh // 2)):
                        MR = write_features(MR, 'feature', df_features[num], ind_MR)
                        if ignore_names == False:
                            MR = write_features(MR, 'name', df_name[num], ind_MR)
                        MR = write_features(MR, 'position', 'part', ind_MR)
                        MR = write_features(MR, 'feature_coordinates', [df_starts[num], df_ends[num]], ind_MR)
                    else:
                        pass
        ind += len(repeats_starts)
    return MR

### RepeatMasker database

<a name='RM'></a>
[Return to Table of Contents](#TOC)

Download https://genome.ucsc.edu/cgi-bin/hgTables?hgsid=1568791217_4daOQLVBcLGDgwRTIvQ0udta7Yd0&clade=mammal&org=Human&db=hs1&hgta_group=genes&hgta_track=hub_3671779_refSeqComposite&hgta_table=0&hgta_regionType=genome&position=chr9%3A145%2C458%2C455-145%2C495%2C201&hgta_outputType=gff&hgta_outFileName=RepeatMasker.gtf

In [12]:
f = gzip.open("../new/Annotation/RepeatMasker.gtf.gz", mode='rb')
column_names = ['seqname', 'start', 'end', 'name', 'score', 'strand', 'i', 'j', 'f']
RM = pd.read_csv(f, sep = "\t")
name = RM['name'].tolist()
feature = [i.split('#')[1] for i in name]
name2 = [i.split('#')[0] for i in name]
RM['feature'] = feature
RM['Name'] = name2
RM = RM[RM['#chrom'] != 'chrM']
RM = RM[RM['strand'] == '+']
RM = RM.sort_values(by=['#chrom', 'thickStart'])
RM = RM.sort_values(
   by="#chrom",
   key=lambda x: np.argsort(index_natsorted(RM["#chrom"]))
)
RM.head()

,#chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,blockSizes,blockStarts,id,description,feature,Name
4163698,chr1,0,2705,(ACCCTA)n#Simple_repeat,29,+,1,2705,0,3,"0,2704,0","-1,1,-1",1,2590 2.9 0.6 1.0 chr1 2 2705 (248384623) + (AC...,Simple_repeat,(ACCCTA)n
4163699,chr1,0,7536,L1MC3#LINE/L1,223,+,4663,7533,0,9,"4912,600,518,158,0,1001,108,392,3","-1,4663,-1,5528,-1,6131,-1,7141,-1",5,1304 19.6 8.0 2.0 chr1 4664 5263 (248382065) +...,LINE/L1,L1MC3
4163703,chr1,5267,5850,MER34C_v#LTR/ERV1,147,+,5274,5528,0,3,"6,254,322","-1,7,-1",6,1403 14.7 2.0 0.8 chr1 5275 5528 (248381800) +...,LTR/ERV1,MER34C_v
4163704,chr1,5685,6131,MSTA1#LTR/ERVL-MaLR,148,+,5686,6131,0,3,"0,445,0","-1,1,-1",7,2442 14.8 5.8 1.3 chr1 5687 6131 (248381197) +...,LTR/ERVL-MaLR,MSTA1
4163705,chr1,8394,8840,MER4A1#LTR/ERV1,106,+,8398,8840,0,3,"3,442,0","-1,4,-1",8,3028 10.6 6.1 0.0 chr1 8399 8840 (248378488) +...,LTR/ERV1,MER4A1


In [14]:
MRpur['feature'] = ['-']*len(MRpur)
MRpur['name'] = ['-']*len(MRpur)
MRpur['position'] = ['-']*len(MRpur)
MRpur['feature_coordinates'] = ['-']*len(MRpur)

MRpur_annotated = annotation(RM, MRpur, chr_list_XY, False)
MRpur_annotated.head()

chr1 chr2 chr3 chr4 chr5 chr6 chr7 chr8 chr9 chr10 chr11 chr12 chr13 chr14 chr15 chr16 chr17 chr18 chr19 chr20 chr21 chr22 chrX chrY 

,chr,start1,start2,end,lsh,dist,first shoulder,second shoulder,repeat_seq,pattern,complexity,repeat_type,feature,name,position,feature_coordinates
0,chr1,50042,50060,50071,11,7,GGAGGGAGGGA,AGGGAGGGAGG,GGAGGGAGGGAGGCAGAAAGGGAGGGAGG,GGAG,4,STR,Low_complexity,G-rich,whole,"[50040, 50125]"
1,chr1,50059,50076,50086,10,7,AAGGGAGGGA,AGGGAGGGAA,AAGGGAGGGAGGCAGAAAGGGAGGGAA,-,27,short_complex,Low_complexity,G-rich,whole,"[50040, 50125]"
2,chr1,50082,50096,50107,11,3,GGAAGGAAGGA,AGGAAGGAAGG,GGAAGGAAGGAGCAAGGAAGGAAGG,GGAA,4,STR,Low_complexity,G-rich,whole,"[50040, 50125]"
3,chr1,50097,50108,50119,11,0,GGAAGGAAGGA,AGGAAGGAAGG,GGAAGGAAGGAAGGAAGGAAGG,GGAA,4,STR,-,-,-,-
4,chr1,73343,73369,73379,10,16,AGAGAGAGAA,AAGAGAGAGA,AGAGAGAGAAAGAGAAAGAAAGAAAGAAGAGAGAGA,AG,2,STR,Simple_repeat,(AGAA)n,part,"[73349, 73514]"


### Genes

<a name='genes'></a>
[Return to Table of Contents](#TOC)

Coordinates of genes in TtoT genome
https://genome.ucsc.edu/cgi-bin/hgTables?hgsid=1479044017_GvxaoDm1LcBPZJGO8X8KD7QsPevb&clade=hub_3267197&org=hub_3267197_human&db=hub_3267197_GCA_009914755.4&hgta_group=genes&hgta_track=hub_3267197_refSeqComposite&hgta_table=0&hgta_regionType=genome&position=CP068269.2%3A145%2C458%2C455-145%2C495%2C201&hgta_outputType=gff&hgta_outFileName=RepeatMasker.gtf

In [15]:
f = gzip.open("genes_T2T-CHM13v2.0.gtf.gz", mode='rb')
column_names = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
genes = pd.read_csv(f, sep = "\t", names=column_names)

# add names of chrs
l = list(range(1,23))
l.reverse()
#without 'chr'
chrom_list_XY = ['X'] + l + ['Y']
chr_list_XY = ['chr'+str(i) for i in chrom_list_XY]
chr_list_XY_GB = []
GB_ind = genes['seqname'].unique()
for seq in range(0, len(GB_ind)):
    df2 = genes[genes['seqname'] == GB_ind[seq]]
    chr_list_XY_GB.extend([chr_list_XY[seq]]*len(df2))
genes['chr'] = chr_list_XY_GB

genes = genes[genes['strand'] == '+']
genes = genes[['chr', 'start', 'end', 'feature']]
genes = genes.sort_values(by=['chr', 'start'])
genes = genes.sort_values(
   by="chr",
   key=lambda x: np.argsort(index_natsorted(genes["chr"]))
)

genes = genes.rename(columns={"chr": "#chrom", "start": "thickStart", "end": "thickEnd"})
genes.head()

,#chrom,thickStart,thickEnd,feature
1620915,chr1,205300,205629,exon
1620917,chr1,205456,205629,exon
1620916,chr1,208474,213723,exon
1620918,chr1,210907,213723,exon
1620919,chr1,246378,247209,exon


In [16]:
chrom_list_XY = list(range(1,23)) + ['X', 'Y']
chr_list_XY = ['chr'+str(i) for i in chrom_list_XY]
MR_annotated_genes = annotation(genes, MRpur_annotated, chr_list_XY, True)
MR_annotated_genes.head()

chr1 chr2 chr3 chr4 chr5 chr6 chr7 chr8 chr9 chr10 chr11 chr12 chr13 chr14 chr15 chr16 chr17 chr18 chr19 chr20 chr21 chr22 chrX chrY 

,chr,start1,start2,end,lsh,dist,first shoulder,second shoulder,repeat_seq,pattern,complexity,repeat_type,feature,name,position,feature_coordinates
0,chr1,50042,50060,50071,11,7,GGAGGGAGGGA,AGGGAGGGAGG,GGAGGGAGGGAGGCAGAAAGGGAGGGAGG,GGAG,4,STR,Low_complexity,G-rich,whole,"[50040, 50125]"
1,chr1,50059,50076,50086,10,7,AAGGGAGGGA,AGGGAGGGAA,AAGGGAGGGAGGCAGAAAGGGAGGGAA,-,27,short_complex,Low_complexity,G-rich,whole,"[50040, 50125]"
2,chr1,50082,50096,50107,11,3,GGAAGGAAGGA,AGGAAGGAAGG,GGAAGGAAGGAGCAAGGAAGGAAGG,GGAA,4,STR,Low_complexity,G-rich,whole,"[50040, 50125]"
3,chr1,50097,50108,50119,11,0,GGAAGGAAGGA,AGGAAGGAAGG,GGAAGGAAGGAAGGAAGGAAGG,GGAA,4,STR,-,-,-,-
4,chr1,73343,73369,73379,10,16,AGAGAGAGAA,AAGAGAGAGA,AGAGAGAGAAAGAGAAAGAAAGAAAGAAGAGAGAGA,AG,2,STR,Simple_repeat,(AGAA)n,part,"[73349, 73514]"


### Introns

<a name='introns'></a>
[Return to Table of Contents](#TOC)

In [19]:
exon = genes[genes['feature']=='exon']
start_intron = []
end_intron = []
chr_intron = []
for Chr in chr_list_XY:
    df2 = exon[exon['#chrom']==Chr]
    starts = df2['thickStart'].tolist()
    ends = df2['thickEnd'].tolist()
    chr_intron.extend([Chr]*(len(starts)-1))
    for j in range(1, len(starts)):
        start_intron.append(ends[j-1])
        end_intron.append(starts[j])
d = {'#chrom': chr_intron, 'thickStart': start_intron, 'thickEnd': end_intron}
intron = pd.DataFrame(data=d)
intron['Len'] = intron['thickEnd']-intron['thickStart']
intron = intron[intron["Len"]>0]
intron = intron[['#chrom', 'thickStart', 'thickEnd']]
intron.to_csv('intron.txt', sep='\t', index=False)
intron['feature'] = ['intron']*len(intron)
MR_annotated_RM_genes_introns = annotation(intron, MR_annotated_genes, chr_list_XY, True)
MR_annotated_RM_genes_introns.head()

,chr,start1,start2,end,lsh,dist,first shoulder,second shoulder,repeat_seq,pattern,complexity,repeat_type,feature,name,position,feature_coordinates
0,chr1,50042,50060,50071,11,7,GGAGGGAGGGA,AGGGAGGGAGG,GGAGGGAGGGAGGCAGAAAGGGAGGGAGG,GGAG,4,STR,Low_complexity,G-rich,whole,"[50040, 50125]"
1,chr1,50059,50076,50086,10,7,AAGGGAGGGA,AGGGAGGGAA,AAGGGAGGGAGGCAGAAAGGGAGGGAA,-,27,short_complex,Low_complexity,G-rich,whole,"[50040, 50125]"
2,chr1,50082,50096,50107,11,3,GGAAGGAAGGA,AGGAAGGAAGG,GGAAGGAAGGAGCAAGGAAGGAAGG,GGAA,4,STR,Low_complexity,G-rich,whole,"[50040, 50125]"
3,chr1,50097,50108,50119,11,0,GGAAGGAAGGA,AGGAAGGAAGG,GGAAGGAAGGAAGGAAGGAAGG,GGAA,4,STR,-,-,-,-
4,chr1,73343,73369,73379,10,16,AGAGAGAGAA,AAGAGAGAGA,AGAGAGAGAAAGAGAAAGAAAGAAAGAAGAGAGAGA,AG,2,STR,Simple_repeat,(AGAA)n,part,"[73349, 73514]"


### Enhancers

<a name='enhancers'></a>
[Return to Table of Contents](#TOC)

In [25]:
column_names = ['#chrom', 'thickStart', 'thickEnd']
enhancer = pd.read_csv('enhancers_T2T.bed', sep='\t', names=column_names)
enhancer['feature'] = ['enhancer']*len(enhancer)
MR_annotated_enhancer = annotation(enhancer, MR_annotated_RM_genes_introns, chr_list_XY, True)
MR_annotated_enhancer.head()

,chr,start1,start2,end,lsh,dist,first shoulder,second shoulder,repeat_seq,pattern,complexity,repeat_type,feature,name,position,feature_coordinates
0,chr1,50042,50060,50071,11,7,GGAGGGAGGGA,AGGGAGGGAGG,GGAGGGAGGGAGGCAGAAAGGGAGGGAGG,GGAG,4,STR,Low_complexity,G-rich,whole,"[50040, 50125]"
1,chr1,50059,50076,50086,10,7,AAGGGAGGGA,AGGGAGGGAA,AAGGGAGGGAGGCAGAAAGGGAGGGAA,-,27,short_complex,Low_complexity,G-rich,whole,"[50040, 50125]"
2,chr1,50082,50096,50107,11,3,GGAAGGAAGGA,AGGAAGGAAGG,GGAAGGAAGGAGCAAGGAAGGAAGG,GGAA,4,STR,Low_complexity,G-rich,whole,"[50040, 50125]"
3,chr1,50097,50108,50119,11,0,GGAAGGAAGGA,AGGAAGGAAGG,GGAAGGAAGGAAGGAAGGAAGG,GGAA,4,STR,-,-,-,-
4,chr1,73343,73369,73379,10,16,AGAGAGAGAA,AAGAGAGAGA,AGAGAGAGAAAGAGAAAGAAAGAAAGAAGAGAGAGA,AG,2,STR,Simple_repeat,(AGAA)n,part,"[73349, 73514]"


### Promoters

<a name='promoters'></a>
[Return to Table of Contents](#TOC)

In [26]:
column_names = ['#chrom', 'thickStart', 'thickEnd']
promoter = pd.read_csv('promoters_T2T.bed', sep='\t', names=column_names)
promoter['feature'] = ['promoter']*len(promoter)
MR_annotated_enhancer_promoter = annotation(promoter, MR_annotated_enhancer, chr_list_XY, True)
MR_annotated_enhancer_promoter.head()

chr1 chr2 chr3 chr4 chr5 chr6 chr7 chr8 chr9 chr10 chr11 chr12 chr13 chr14 chr15 chr16 chr17 chr18 chr19 chr20 chr21 chr22 chrX chrY 

,chr,start1,start2,end,lsh,dist,first shoulder,second shoulder,repeat_seq,pattern,complexity,repeat_type,feature,name,position,feature_coordinates
0,chr1,50042,50060,50071,11,7,GGAGGGAGGGA,AGGGAGGGAGG,GGAGGGAGGGAGGCAGAAAGGGAGGGAGG,GGAG,4,STR,Low_complexity,G-rich,whole,"[50040, 50125]"
1,chr1,50059,50076,50086,10,7,AAGGGAGGGA,AGGGAGGGAA,AAGGGAGGGAGGCAGAAAGGGAGGGAA,-,27,short_complex,Low_complexity,G-rich,whole,"[50040, 50125]"
2,chr1,50082,50096,50107,11,3,GGAAGGAAGGA,AGGAAGGAAGG,GGAAGGAAGGAGCAAGGAAGGAAGG,GGAA,4,STR,Low_complexity,G-rich,whole,"[50040, 50125]"
3,chr1,50097,50108,50119,11,0,GGAAGGAAGGA,AGGAAGGAAGG,GGAAGGAAGGAAGGAAGGAAGG,GGAA,4,STR,-,-,-,-
4,chr1,73343,73369,73379,10,16,AGAGAGAGAA,AAGAGAGAGA,AGAGAGAGAAAGAGAAAGAAAGAAAGAAGAGAGAGA,AG,2,STR,Simple_repeat,(AGAA)n,part,"[73349, 73514]"


In [27]:
MR_annotated_enhancer_promoter.to_csv('MRpur_annotated.csv', sep='\t', index=False)

### File for heatmap with contributions of each features

<a name='contribution'></a>
[Return to Table of Contents](#TOC)

In [28]:
genome_len = 3117275501

calculating number and percent of repeats by type and feature

In [30]:
MR_annotated = pd.read_csv('MRpur_annotated.csv', sep='\t')
MRsegm_annotated = pd.read_csv('MRpur_inAGsegm_annotated.csv', sep='\t')

MR_by_repeattype_numb = {}
MRsegm_by_repeattype_numb = {}
keys1 = list(MR_annotated.repeat_type.unique())
keys2 = list(MRsegm_annotated.repeat_type.unique())
for key in keys1:
    number = len(MR_annotated[MR_annotated['repeat_type'] == key])
    MR_by_repeattype_numb[key] = number
for key in keys2:
    number = len(MRsegm_annotated[MRsegm_annotated['repeat_type'] == key])
    MRsegm_by_repeattype_numb[key] = number
print(MR_by_repeattype_numb)
print(MRsegm_by_repeattype_numb)

{'STR': 183316, 'short_complex': 63267, 'long_TR': 5905, 'long_complex': 154}
{'STR': 66031, 'short_complex': 19695, 'long_TR': 5571, 'long_complex': 102}


In [31]:
df = MR_annotated.replace('-', 'Unknown')
df['feature'] = df.feature.str.split('; ')
df = df.explode("feature")
df[['feature1', 'feature2']] = df.feature.str.split("/", expand = True)
df = df.reset_index(drop=True)
df = df.reset_index()
df = df.groupby(['repeat_type', 'feature1'])['index'].count()
df = df.to_frame().reset_index()
df = df.rename(columns={"index": "number", 'feature1':'feature'})

# add per cent
number_per_cent = []
for row in range(0, len(df)):
    repeat_type = df.iloc[row]['repeat_type']
    num = df.iloc[row]['number']
    num_repeatType = MR_by_repeattype_numb[repeat_type]
    number_per_cent.append((num / num_repeatType)*100)
    
df['number_pc'] = number_per_cent
    
print(df.feature.unique())
df = df.replace('DNA', 'DNA transposons')
df = df.replace('RC', 'RC transposons')
df = df.replace('Retroposon', 'SVA')

df.head()

['CDS' 'DNA' 'LINE' 'LTR' 'Low_complexity' 'RC' 'Retroposon' 'SINE'
 'Satellite' 'Simple_repeat' 'Unknown' 'enhancer' 'exon' 'intron'
 'promoter' 'scRNA' 'srpRNA']


,repeat_type,feature,number,number_pc
0,STR,CDS,78,0.042549
1,STR,DNA transposons,225,0.122739
2,STR,LINE,1865,1.017369
3,STR,LTR,758,0.413494
4,STR,Low_complexity,30376,16.570294


In [32]:
df2 = MRsegm_annotated.replace('-', 'Unknown')
df2['feature'] = df2.feature.str.split('; ')
df2 = df2.explode("feature")
df2[['feature1', 'feature2']] = df2.feature.str.split("/", expand = True)
df2 = df2.reset_index(drop=True)
df2 = df2.reset_index()
df2 = df2.groupby(['repeat_type', 'feature1'])['index'].count()
df2 = df2.to_frame().reset_index()
df2 = df2.rename(columns={"index": "number", 'feature1':'feature'})

# add per cent
number_per_cent = []
for row in range(0, len(df2)):
    repeat_type = df2.iloc[row]['repeat_type']
    num = df2.iloc[row]['number']
    num_repeatType = MRsegm_by_repeattype_numb[repeat_type]
    number_per_cent.append((num / num_repeatType)*100)
    
df2['number_pc'] = number_per_cent

print(df2.feature.unique())
df2 = df2.replace('DNA', 'DNA transposons')
df2 = df2.replace('RC', 'RC transposons')
df2 = df2.replace('Retroposon', 'SVA')
df2.head()

['CDS' 'DNA' 'LINE' 'LTR' 'Low_complexity' 'SINE' 'Satellite'
 'Simple_repeat' 'Unknown' 'enhancer' 'exon' 'intron' 'promoter' 'srpRNA']


,repeat_type,feature,number,number_pc
0,STR,CDS,17,0.025745
1,STR,DNA transposons,23,0.034832
2,STR,LINE,297,0.449789
3,STR,LTR,83,0.125699
4,STR,Low_complexity,22046,33.387348


In [33]:
df3 = df.rename(columns={"number": "number_allMRpur", 'number_pc': 'number_allMRpur_pc'})
df2 = df2.rename(columns={"number": "number_MRpur_inAGsegm", 'number_pc': 'number_MRpur_inAGsegm_pc'})
df3 = df3.sort_values(by=['repeat_type', 'feature'])
df3['number_MRpur_inAGsegm'] = [0]*len(df3)
df3['number_MRpur_inAGsegm_pc'] = [0]*len(df3)
for i in range(0, len(df3)):
    repeat_type = df3.iloc[i]['repeat_type']
    feature = df3.iloc[i]['feature']
    df4 = df2[df2['repeat_type'] == repeat_type]
    if feature in df4.feature.tolist():
        row = df4[df4['feature']==feature]
        df3.at[i, 'number_MRpur_inAGsegm'] = row.iloc[0]['number_MRpur_inAGsegm']
        df3.at[i, 'number_MRpur_inAGsegm_pc'] = row.iloc[0]['number_MRpur_inAGsegm_pc']
df3.head()

,repeat_type,feature,number_allMRpur,number_allMRpur_pc,number_MRpur_inAGsegm,number_MRpur_inAGsegm_pc
0,STR,CDS,78,0.042549,17,0.025745
1,STR,DNA transposons,225,0.122739,23,0.034832
2,STR,LINE,1865,1.017369,297,0.449789
3,STR,LTR,758,0.413494,83,0.125699
4,STR,Low_complexity,30376,16.570294,22046,33.387348


adding contributions of each feature

In [34]:
RM_contribution = RM.replace('-', 'Unknown')
RM_contribution[['feature1', 'feature2']] = RM_contribution.feature.str.split("/", expand = True)
RM_contribution['Len'] = RM_contribution["thickEnd"] - RM_contribution['thickStart']
RM_contribution = RM_contribution[['feature1', 'Len']]
RM_contribution = RM_contribution.groupby(['feature1'])['Len'].sum()
RM_contribution = RM_contribution.to_frame().reset_index()
RM_contribution = RM_contribution.replace('DNA', 'DNA transposons')
RM_contribution = RM_contribution.replace('RC', 'RC transposons')
RM_contribution = RM_contribution.replace('Retroposon', 'SVA')
RM_contribution = RM_contribution.rename(columns={'feature1':'feature'})
print(RM_contribution.feature.unique())
proportion = []
for i in range(0, len(RM_contribution)):
    proportion.append(RM_contribution.iloc[i]['Len']/genome_len)
RM_contribution['proportion'] = proportion
RM_contribution.head()

['DNA transposons' 'LINE' 'LTR' 'Low_complexity' 'RC transposons'
 'Retroposon' 'SINE' 'Satellite' 'Simple_repeat' 'Unknown' 'rRNA' 'scRNA'
 'snRNA' 'srpRNA' 'tRNA']


,feature,Len,proportion
0,DNA transposons,62872991,0.020169
1,LINE,464215444,0.148917
2,LTR,174930907,0.056117
3,Low_complexity,6578930,0.002110
4,RC transposons,314210,0.000101


In [35]:
#genes
genes['Len'] = genes["thickEnd"] - genes['thickStart']
genes_contribution = genes.groupby(['feature'])['Len'].sum()
genes_contribution = genes_contribution.to_frame().reset_index()
print(genes_contribution.feature.unique())
proportion = []
for i in range(0, len(genes_contribution)):
    proportion.append(genes_contribution.iloc[i]['Len']/genome_len)
genes_contribution['proportion'] = proportion
genes_contribution.head()

['CDS' 'exon' 'start_codon' 'stop_codon']


,feature,Len,proportion
0,CDS,56116925,0.018002
1,exon,176677617,0.056677
2,start_codon,66051,0.000021
3,stop_codon,66064,0.000021


In [36]:
#introns
intron['Len'] = intron["thickEnd"] - intron['thickStart']
intron_sum = intron['Len'].sum()
intron_contribution = intron_sum/genome_len
intron_contribution

0.9779912494170018

In [38]:
#enhancers
enhancer['Len'] = enhancer["thickEnd"] - enhancer['thickStart']
enhancer_sum = enhancer['Len'].sum()
enhancer_contribution = enhancer_sum/genome_len
enhancer_contribution

0.051793982902122705

In [39]:
#promoters
promoter['Len'] = promoter["thickEnd"] - promoter['thickStart']
promoter_sum = promoter['Len'].sum()
promoter_contribution = promoter_sum/genome_len
promoter_contribution

0.019874883365337815

In [40]:
d = {'feature': ['intron', 'promoter', 'enhancer'], 'Len': [intron_sum, promoter_sum, enhancer_sum], 'proportion': [intron_contribution, promoter_contribution, enhancer_contribution]}
result = pd.DataFrame(data=d)

data = [genes_contribution, RM_contribution, result]
contributions = pd.concat(data)
contributions.head()

,feature,Len,proportion
0,CDS,56116925,0.018002
1,exon,176677617,0.056677
2,start_codon,66051,0.000021
3,stop_codon,66064,0.000021
0,DNA transposons,62872991,0.020169


In [41]:
df3 = df3[df3['feature']!='Unknown']
numb_allMR_plusFeatureInput = []
numb_MRsegm_plusFeatureInput = []
for i in range(0, len(df3)):
    feature = df3.iloc[i]['feature']
    df = contributions[contributions["feature"]==feature]
    contribution = df.iloc[0]['proportion']
    numb_allMR_plusFeatureInput.append((df3.iloc[i]['number_allMRpur_pc'])/contribution)
    numb_MRsegm_plusFeatureInput.append((df3.iloc[i]['number_MRpur_inAGsegm_pc'])/contribution)
df3['numb_allMR_plusFeatureInput'] = numb_allMR_plusFeatureInput
df3['numb_MRsegm_plusFeatureInput'] = numb_MRsegm_plusFeatureInput
#df.to_csv('num_MRrep_featuresContributions.csv', sep='\t', index=False)
df3.head()

,repeat_type,feature,number_allMRpur,number_allMRpur_pc,number_MRpur_inAGsegm,number_MRpur_inAGsegm_pc,numb_allMR_plusFeatureInput,numb_MRsegm_plusFeatureInput
0,STR,CDS,78,0.042549,17,0.025745,2.363609,1.430153
1,STR,DNA transposons,225,0.122739,23,0.034832,6.085457,1.726995
2,STR,LINE,1865,1.017369,297,0.449789,6.831783,3.020398
3,STR,LTR,758,0.413494,83,0.125699,7.368473,2.239953
4,STR,Low_complexity,30376,16.570294,22046,33.387348,7851.454762,15819.831357


### Transposons

<a name='TE'></a>
[Return to Table of Contents](#TOC)

In [ ]:
transposons = ['SINE', 'LTR', 'LINE', 'DNA', 'Retroposon', 'RC']

MR_annotated = pd.read_csv('data/outputs/MRpur_annotated.csv', sep='\t')
MRsegm_annotated = pd.read_csv('data/outputs/MRpur_inAGsegm_annotated.csv', sep='\t')

df = MR_annotated.replace('-', 'Unknown')
df['feature'] = df.feature.str.split('; ')
df = df.explode("feature")
df[['feature1', 'feature2']] = df.feature.str.split("/", expand = True)
df = df.reset_index(drop=True)
df = df.reset_index()
MR_transposons = df[df['feature1'].isin(transposons)]

df2 = MRsegm_annotated.replace('-', 'Unknown')
df2['feature'] = df2.feature.str.split('; ')
df2 = df2.explode("feature")
df2[['feature1', 'feature2']] = df2.feature.str.split("/", expand = True)
df2 = df2.reset_index(drop=True)
df2 = df2.reset_index()
MRsegm_transposons = df2[df2['feature1'].isin(transposons)]

MR_transposons = MR_transposons.groupby(['lsh'])['index'].count()
MR_transposons = MR_transposons.to_frame().reset_index()
MR_transposons.to_csv('data/outputs/transposons_number.csv', sep='\t', index=False)
MRsegm_transposons = MRsegm_transposons.groupby(['lsh'])['index'].count()
MRsegm_transposons = MRsegm_transposons.to_frame().reset_index()
MRsegm_transposons.to_csv('data/outputs/transposons_segm_number.csv', sep='\t', index=False)

### ISTAT

<a name='istat'></a>
[Return to Table of Contents](#TOC)
- finding the most common features for checking overlaps significance  

In [42]:
df = pd.read_csv('MRpur_annotated.csv', sep='\t')
df['feature'] = df.feature.str.split('; ')
df = df.explode("feature")
df['name'] = df.name.str.split('; ')
df2 = df.explode("name")
df2[['feature1', 'feature2']] = df2.feature.str.split("/", expand = True)
df2 = df2.reset_index(drop=True)
df2 = df2.reset_index()
df2 = df2[~df2['feature1'].isin(['Simple_repeat', 'Low_complexity', 'DNA', 'srpRNA', 'scRNA', 'Satellite', '-', 'Unknown', 'RC'])]
df2 = df2[df2['name']!='-']
df3 = df2.groupby(['repeat_type', 'feature1', 'feature2', 'name'])['index'].count()
df3 = df3.to_frame().reset_index()
df3 = df3.rename(columns={"index": "number", 'feature1':'class', 'feature2':'family'})
df3 = df3.sort_values(by=['repeat_type', 'number'], ascending=False)
df3 = df3
df3.head()

,repeat_type,class,family,name,number
642,short_complex,SINE,Alu,AluSx,3514
627,short_complex,SINE,Alu,AluJb,1988
648,short_complex,SINE,Alu,AluY,1476
643,short_complex,SINE,Alu,AluSx1,625
646,short_complex,SINE,Alu,AluSz,527


## Conservation

<a name='conservation'></a>
[Return to Table of Contents](#TOC)

conservation score phyloP for multiple alignment by chromosomes http://hgdownload.soe.ucsc.edu/goldenPath/hg38/phyloP30way/

In [ ]:
# function for reading wigFix files
flatten = lambda l: [item for sublist in l for item in sublist]
def read_wig_fixed(file):
    phylo = pd.read_csv(file, sep = ',', compression = 'gzip', header = None, dtype = str)
    phylo_index_entries = phylo.loc[[entry.startswith('fixedStep') for entry in phylo[0].astype(str)]][0].str.split('start=', expand = True)[1].str.split(' ', expand = True)[0].astype(int)
    phylo_index_end = list(phylo_index_entries.index[1:]) + [phylo.index.max()]
    phylo_new_index = flatten([list(range(entry-1, entry+(end-start)-1)) for start, end, entry in zip(phylo_index_entries.index, phylo_index_end, phylo_index_entries)]) + list(phylo_index_entries[-1:])
    phylo.index = phylo_new_index
    phylo = pd.to_numeric(phylo[0], errors='coerce').dropna()
    phylo = phylo.loc[~phylo.index.duplicated()].sort_index()
    return phylo

def main():
    for Chr in chr_list_XY:
        print(Chr, end=' ')
        infile = 'conservation/'+Chr+'.phyloP30way.wigFix.gz'
        outfile = 'conservation/'+Chr+'_phylo.txt'

        df = read_wig_fixed(infile)
        df.to_csv(outfile, sep='\t')
                

if __name__ == '__main__':
    main()

In [ ]:
# MR is file with repeats where names of columns are chr, start of repeat, end of repeat
MR = pd.read_csv('data/MR_hg38.bed', sep='\t', names=['chr', 'start', 'end'])
MR['length'] = MR['end'] - MR['start']
score_list = []

for Chr in chr_list_XY:
    print(Chr, end=' ')
    MR_chr = MR[MR['chr'] == Chr]
    MR_chr = MR_chr.reset_index()
    
    infile = 'conservation/' + Chr + '_phylo.txt'
    df = pd.read_csv(infile, sep='\t', names = ['position', 'score'])
    
    for row in range(0, len(MR_chr)):
        start = MR_chr.iloc[row]['start']
        end = MR_chr.iloc[row]['end']
        length = MR_chr.iloc[row]['length']
        score_sum = 0
        for n in range(start, end):
            try:
                score_sum += df.at[n, 'score']
            except:
                length -= 1
                continue
        if length == 0:
            score_list.append(0)
        else:
            score_list.append(score_sum/length)

MR['conservation_score'] = score_list
MR.to_csv('data/conservation_score_MR.csv', sep='\t', index=False)

In [ ]:
# average conservation
mean_score = 0
for Chr in chr_list_XY:
    print(Chr, end=' ')
    infile = '../new/conservation/' + Chr + '_phylo.txt'
    df = pd.read_csv(infile, sep='\t', names = ['position', 'score'])
    
    score_sum = df.sum()['score']
    mean_score += score_sum/len(df)
    
mean_score = mean_score/len(chr_list_XY)
        
print(mean_score)